# Metaprogramming

A "meta" program is a program that manipulates programs.

Most common use refers to a program that *generates* another program.

In the last class we saw how Julia can generate specialized code for you.

Sometimes that's not enough, and you need to write a program to explicitly generate the code needed for a specialized problem.

Julia allows us to talk in a "meta" way ("one level up"), about Julia code,  that is to "treat code as data" and manipulate it as just another object in Julia. (This is very similar to Lisp.)

## Symbols and Expressions

One of the most basic objects in this approach are unevaluated *symbols*:

In [1]:
:a  # "the symbol a"

:a

In [2]:
typeof(:a)

Symbol

`:a` refers to the symbol `a`. We can evaluate it with the `eval` function:

In [3]:
eval(:a)

LoadError: [91mUndefVarError: a not defined[39m

`a` must be defined for this to work:

In [4]:
a = 3

3

In [5]:
eval(:a)

3

The `eval` function takes an expression and evaluates it, that is, *generates the corresponding code*

Operators and function names are also symbols:

In [6]:
:+, :sin

(:+, :sin)

In [7]:
typeof(:+)

Symbol

Symbols may be combined into *expressions*, which are the basic objects that represent pieces of Julia code:

In [8]:
ex = :(a + b)  # the expression 'a + b'

:(a + b)

In [9]:
typeof(ex)

Expr

In [10]:
b = 7
eval(ex)

10

An expression is just a Julia object, so we can introspect (find out information about it):

In [11]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol a
    3: Symbol b
  typ: Any


In [12]:
dump(:(x = 3))

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol x
    2: Int64 3
  typ: Any


In [13]:
# ex.<TAB>

In [14]:
ex.head

:call

In [15]:
ex.args

3-element Array{Any,1}:
 :+
 :a
 :b

The job of Julia's parser is to convert a sequence of characters into these `Expr` objects:

In [16]:
parse("a + b")

:(a + b)

More complicated expressions are represented as "abstract syntax trees" (ASTs), consisting of expressions nested inside expressions:

In [17]:
ex = :( sin(3a + 2b^2) )

:(sin(3a + 2 * b ^ 2))

In [18]:
ex.args

2-element Array{Any,1}:
 :sin             
 :(3a + 2 * b ^ 2)

In [19]:
typeof(ex.args[2])

Expr

In [20]:
ex.args[2].args

3-element Array{Any,1}:
 :+          
 :(3a)       
 :(2 * b ^ 2)

In [21]:
ex.args[1] = :cos

:cos

In [22]:
ex

:(cos(3a + 2 * b ^ 2))

In [23]:
blk = quote
    println("Hello")
end

quote  # In[23], line 2:
    println("Hello")
end

In [24]:
eval(blk)

Hello


In [25]:
push!(blk.args, :(println("AFTER")))

3-element Array{Any,1}:
 :( # In[23], line 2:)
 :(println("Hello"))  
 :(println("AFTER"))  

In [26]:
blk

quote  # In[23], line 2:
    println("Hello")
    println("AFTER")
end

In [27]:
eval(blk)

Hello
AFTER


In [28]:
unshift!(blk.args, :(println("BEFORE")))

4-element Array{Any,1}:
 :(println("BEFORE")) 
 :( # In[23], line 2:)
 :(println("Hello"))  
 :(println("AFTER"))  

In [29]:
eval(blk)

BEFORE
Hello
AFTER


Expressions can be arbitrary Julia code that when evaluated will have side effects. For longer blocks of code, `quote...end` may be used instead of `:( ... )`

In [30]:
ex2 = 
quote
    y = 3
    z = sin(y+1)
end

quote  # In[30], line 3:
    y = 3 # In[30], line 4:
    z = sin(y + 1)
end

In [31]:
eval(ex2)
y

3

In [32]:
eval(ex2)
z

-0.7568024953079282

The full form of the abstract syntax tree in a style similar to a Lisp s-expression can be obtained using functions from the `Meta` module in `Base`:

In [33]:
Meta.show_sexpr(ex2)

(:block,
  (:line, 3, Symbol("In[30]")),
  (:(=), :y, 3),
  (:line, 4, Symbol("In[30]")),
  (:(=), :z, (:call, :sin, (:call, :+, :y, 1)))
)

Another way of seeing the structure is with `dump`:

In [34]:
dump(ex2)

Expr
  head: Symbol block
  args: Array{Any}((4,))
    1: Expr
      head: Symbol line
      args: Array{Any}((2,))
        1: Int64 3
        2: Symbol In[30]
      typ: Any
    2: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol y
        2: Int64 3
      typ: Any
    3: Expr
      head: Symbol line
      args: Array{Any}((2,))
        1: Int64 4
        2: Symbol In[30]
      typ: Any
    4: Expr
      head: Symbol =
      args: Array{Any}((2,))
        1: Symbol z
        2: Expr
          head: Symbol call
          args: Array{Any}((2,))
            1: Symbol sin
            2: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol +
                2: Symbol y
                3: Int64 1
              typ: Any
          typ: Any
      typ: Any
  typ: Any


## Interpolation

Values can be *interpolated* into expressions with `$`.

In [35]:
x = 2

2

In [36]:
:(x + $x)

:(x + 2)

In [37]:
x = :y

:y

In [38]:
:(x + $x)

:(x + y)

### Why not strings?

*Abstract* syntax vs. *concrete* syntax.

In [39]:
a = "x + y"

"x + y"

In [40]:
b = "z"

"z"

In [41]:
expr = "$a * z"

"x + y * z"

Even worse, what if `z == ")"`?

With `Expr` objects, every expression is well-formed and each slot has its abstract meaning.

### Splat interpolation

In [42]:
vars = [:a, :b, :c]

3-element Array{Symbol,1}:
 :a
 :b
 :c

In [43]:
:(1 - f(0, $(vars...)))

:(1 - f(0, a, b, c))

### `@eval` for interpolating on the fly

You will sometimes see `@eval expr`, which is shorthand for:

```
eval(quote
         expr
     end)
```

allowing items to be interpolated immediately before executing.

In [44]:
fnames = [ Symbol("func$i") for i=1:2]

2-element Array{Symbol,1}:
 :func1
 :func2

In [45]:
for i = 1:length(fnames)
    @eval ($(fnames[i]))(x) = $i
end

In [46]:
func1(0)

1

In [47]:
func2(0)

2

## Macros

With the ability to think of code in terms of a data structure in the Julia language, we can now *manipulate* those data structures, allowing us to *create Julia code from within Julia*.

*Macros* provide a particular use pattern of metaprogramming: replacing one expression with another, in-place, right after parsing.

The [Julia manual](http://julia.readthedocs.org/en/latest/manual/metaprogramming/) puts it like this: 

> macros map a tuple of argument *expressions* to a returned
> *expression*

Macros are useful in several cases:

- to provide a specific notation different than what can normally be written in the language
    - e.g. https://github.com/JuliaOpt/JuMP.jl/blob/release-0.18/examples/sudoku.jl#L44
- to rearrange or delay evaluation of code
- to eliminate boilerplate (repetitive) code
- to automatically generate complex code that would be painful by hand
- to unroll loops for efficiency
- to inline code for efficiency



Macros are invoked using the `@` sign, e.g.

In [48]:
@time sin(10)

  0.000297 seconds (84 allocations: 6.107 KiB)


-0.5440211108893698

A trivial example of defining a macro is the following, which runs whatever code it is passed two times.

In [49]:
macro twice(ex)
    quote
        $ex
        $ex
    end
end

@twice (macro with 1 method)

In [50]:
x = 0
@twice println(x += 1)

LoadError: [91msyntax: invalid assignment location "Main.x"[39m

In [51]:
ex = :(@twice println(x += 1))

:(@twice println(x += 1))

In [52]:
eval(ex)

LoadError: [91msyntax: invalid assignment location "Main.x"[39m

In [53]:
typeof(ex)

Expr

We can see what effect the macro actually has using `macroexpand`:

In [54]:
macroexpand(:(@twice println(x += 1)))

quote  # In[49], line 3:
    (Main.println)(Main.x += 1) # In[49], line 4:
    (Main.println)(Main.x += 1)
end

In [55]:
macroexpand(:(@time sin(10)))

quote  # util.jl, line 235:
    local #6#stats = (Base.gc_num)() # util.jl, line 236:
    local #8#elapsedtime = (Base.time_ns)() # util.jl, line 237:
    local #7#val = sin(10) # util.jl, line 238:
    #8#elapsedtime = (Base.time_ns)() - #8#elapsedtime # util.jl, line 239:
    local #9#diff = (Base.GC_Diff)((Base.gc_num)(), #6#stats) # util.jl, line 240:
    (Base.time_print)(#8#elapsedtime, #9#diff.allocd, #9#diff.total_time, (Base.gc_alloc_count)(#9#diff)) # util.jl, line 242:
    #7#val
end

In [56]:
macro mytime(ex)
    quote
        t0 = time()
        val = $ex
        t1 = time()
        println("$(t1-t0) seconds elapsed")
        val
    end
end

@mytime (macro with 1 method)

In [57]:
@mytime (sleep(1); "done")

1.005418062210083 seconds elapsed


"done"

A line break terminates macro arguments, so blocks need to be passed with `begin...end`:

In [58]:
@time begin
    # code
end

  0.000001 seconds (3 allocations: 144 bytes)


----
**Exercise**: Define a macro `@until` that does an `until` loop.

----

### Macro Hygiene

In [59]:
macro set_x(val)
    :(x = $val)
end

@set_x (macro with 1 method)

In [60]:
@set_x 10

10

In [61]:
x

0

In [62]:
macro set_x(val)
    :($(esc(:x)) = $val)
end

@set_x (macro with 1 method)

In [63]:
@set_x 10

10

In [64]:
x

10

### Macros for numerical performance: Horner's method

There are many interesting examples of macros in `Base`. One that is accessible is Horner's method for evaluating a polynomial:

$$p(x) = a_n x^n + a_{n-1} x^{n-1} + \cdots + a_1 x^1 + a_0$$

may be evaluated efficiently as

$$p(x) = a_0 + x(a_1 + \cdots x(a_{n-2} + \cdots + x(a_{n-1} + x a_n) \cdots ) ) $$
with only $n$ multiplications.

The obvious way to do this is with a `for` loop. But if we know the polynomial *at compile time*, this loop may be unrolled using metaprogramming. This is implemented in the `Math` module in `math.jl` in `Base`, so the name of the macro (which is not exported) is `@Base.Math.horner`.

In [65]:
# copied from base/math.jl
macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :( $(esc(p[i])) + t * $ex )  # actually uses `muladd`
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

@horner (macro with 1 method)

This is called as follows: to evaluate the polynomial $p(x) = 2 + 3x + 4x^2$ at $x=3$, we do

In [66]:
x = 3
@horner(x, 2, 3, 4, 5)

182

To see what the macro does to this call, we again use `macroexpand`:

In [67]:
macroexpand(:(@horner(x, 2, 3, 4, 5, 6, 7, 8, 9, 10)))

quote 
    #19#t = x
    2 + #19#t * (3 + #19#t * (4 + #19#t * (5 + #19#t * (6 + #19#t * (7 + #19#t * (8 + #19#t * (9 + #19#t * 10)))))))
end

In [68]:
macroexpand(:(@Base.Math.horner(x, 2, 3, 4, 5)))

quote 
    #20#t = x
    (Base.Math.muladd)(#20#t, (Base.Math.muladd)(#20#t, (Base.Math.muladd)(#20#t, 5, 4), 3), 2)
end

In [69]:
?muladd

search: muladd



```
muladd(x, y, z)
```

Combined multiply-add, computes `x*y+z` in an efficient manner. This may on some systems be equivalent to `x*y+z`, or to `fma(x,y,z)`. `muladd` is used to improve performance. See [`fma`](@ref).

# Example

```jldoctest
julia> muladd(3, 2, 1)
7

julia> 3 * 2 + 1
7
```


In [70]:
?fma

search: fma findmax findmax! DateFormat @fastmath @dateformat_str UniformScaling



```
fma(x, y, z)
```

Computes `x*y+z` without rounding the intermediate result `x*y`. On some systems this is significantly more expensive than `x*y+z`. `fma` is used to improve accuracy in certain algorithms. See [`muladd`](@ref).


In [71]:
f(x) = Base.Math.@horner(x, 1.2, 2.3, 3.4, 4.5)

f (generic function with 1 method)

In [72]:
@code_llvm f(0.1)


define double @julia_f_63335(double) #0 !dbg !5 {
top:
  %1 = call double @llvm.fmuladd.f64(double %0, double 4.500000e+00, double 3.400000e+00)
  %2 = call double @llvm.fmuladd.f64(double %0, double %1, double 2.300000e+00)
  %3 = call double @llvm.fmuladd.f64(double %0, double %2, double 1.200000e+00)
  ret double %3
}


In [73]:
@code_native f(0.1)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[71]
	pushl	%ebp
	decl	%eax
	movl	%esp, %ebp
	decl	%eax
	movl	$210231424, %eax        ## imm = 0xC87E080
	addl	%eax, (%eax)
	addb	%al, (%eax)
	movsd	(%eax), %xmm1           ## xmm1 = mem[0],zero
Source line: 1
	mulsd	%xmm0, %xmm1
	decl	%eax
	movl	$210231432, %eax        ## imm = 0xC87E088
	addl	%eax, (%eax)
	addb	%al, (%eax)
	addsd	(%eax), %xmm1
	mulsd	%xmm0, %xmm1
	decl	%eax
	movl	$210231440, %eax        ## imm = 0xC87E090
	addl	%eax, (%eax)
	addb	%al, (%eax)
	addsd	(%eax), %xmm1
	mulsd	%xmm1, %xmm0
	decl	%eax
	movl	$210231448, %eax        ## imm = 0xC87E098
	addl	%eax, (%eax)
	addb	%al, (%eax)
	addsd	(%eax), %xmm0
	popl	%ebp
	retl
	nop
	nop
	nop
	nop
	nop
	nop


## `@generated` Functions

Another form of metaprogramming available in Julia.

Generate code based on *types* instead of the input expressions.

In [74]:
@generated function nloops(a::Array{T,N}) where {T,N}
    # inside here,
    # - `a` refers to the type of the argument
    # - `T` and `N` have their usual values
    # - return an expression to compute the answer for this type
end

nloops (generic function with 1 method)